<h1> Intercambio de Llaves Diffie Hellman y El Gamal

_________

Los siguientes cifrados, son alternativas al cifrado RSA para el intercambio de Llaves. La seguridad de ambos cifrados
radica principalmente en el dificil cálculo del logaritmo discreto.

El primero, el cifrado de Diffie - Hellman, fue establecido por Whitfield Diffie y Martin Hellman. Consiste en lo siguiente: ($Z_p$)
         
1. Alicia escoge un grupo $Z_p$ y un generador $<g>$, igualmente escoge una llave privada a. 
2. Alicia cálcula $A\ \equiv \ g^a\ mod\ p$ y le manda A a Bob.
3. Bob teniendo su llave privada $(b)$ y teniendo $g$ y $Z_p$, los cuales son públicos, cálcula $\ B \equiv g^b\ mod\ p$.
4. Bob le manda $B$ a alicia.
5. Alicia hace $B^a=g^{ab}$ y bob hace $A^b=g^{ab}$.
6. Ambos obtienen la misma llave.


<h2> Ejemplo: (Intercambiemos Llaves)


![Figure 1](./img/DH.png)

Alicia escoge $g=13$ en $Z_{37124508045065437}$.

Escoge su llave privada $a=12736871092846$

In [2]:
a=12736871092846
g=13
p=37124508045065437
print ("a =",a)
print ("g =",g)
print ("p =",p)

a = 12736871092846
g = 13
p = 37124508045065437


Alicia calcula $\ A \equiv \ g^a\ mod\ p$ y se lo manda a Bob.

In [3]:
A=pow(g,a,p)
print ("A =",A)

A = 30384510001166445


Bob escoge su llave privada $b=8763247901223$, calcula $\ B \equiv \ g^b\ mod\ p$ y se lo manda a Alicia

In [4]:
b=87632478723934
B=pow(g,b,p)
print ("B =",B)

B = 20299696698362165


Alicia calcula $\ B^a\ mod\ p$ y Bob calcula $\ A^b\ mod\ p$.


In [5]:
llave_alicia=pow(B,a,p)
llave_bob=pow(A,b,p)
print ("Llave Alicia =",llave_alicia)
print ("Llave Bob =",llave_bob)

Llave Alicia = 4908577199504481
Llave Bob = 4908577199504481


Ambos consiguen la misma llave y pueden empezar a encriptar con llave privada.


__________


<h2> El Gamal

Para el segundo cifrado para intercambio de llaves el proceso es un poco más complejo. El cifrado el Gamal a diferencia del cifrado Diffie Hellman es asimétrico, lo cual incrementa su grado de complejidad, además de que este cifrado puede igualmente ser utilizado para el cifrado de texto. Consiste en lo siguiente:

1. Alicia al igual que en Diffie Hellman escoge $g, p, a$ (su llave aleatoria privada), y calcula $\ A \equiv \ g^a\ mod\ p$.
2. Hace públicas $g$ y $p$ y le manda $A$ a Bob.
3. Ahora bob escoge un número $K$ al y un texto $(M)$ que pertenezca al grupo.
4. Bob va a calcular dos variables extras ($c_1$ y $c_2$) de la siguiente manera:
      $\ c_1 \equiv \ g^K\ mod\ p$  y 
      $\ c_2 = M*A^K$
5. Bob envía a Alicia $c_1$ y $c_2$.
6. Alicia define $\ x \equiv c_1^a\ mod\ p$.
7. Alicia calcula $\ x^{-1}*c_2 \equiv c_1^{-a}*M*A^K \equiv g^{-ka}*M*g^{ak} \equiv \ M\ mod\ p$

![Figure 2](./img/ElGamal.png)
     

<h3> Ejemplo (intercambio de texto)

Alicia escoge igual que en el ejemplo anterior $g =\ 13$, $p=\ 633910111$ y a distinto $a =\ 71682723$, y calcula $A$

In [1]:
a=71682723
g=13
p=633910111
A=pow(g,a,p)
print ("a =",a)
print ("g =",g)
print ("p =",p)
print ("A =",A)

a = 71682723
g = 13
p = 633910111
A = 6817155


Bob escoge $K =\ 9821640917248672790347910$ y calcula $c_1$ y $c_2$. Igualmente escoge el texto $M =\ 546780193$. Recibe A de Alicia. Bob le manda $c_1$ y $c_2$ a Alicia.

In [4]:
K=9821640917248672790347910
M='HOLA'
M= int(''.join(str(ord(letra)) for letra in M))
c1=pow(g,K,p)
c2=M*pow(A,K,p)
print ("Texto =",M)
print ("c1 =",c1)
print ("c2 =",c2)

Texto = 72797665
c1 = 207679210
c2 = 18136754276695615


Alicia define $X =\ c_1^a\ mod\ p$ y calcula $X^{-c_2}$

In [5]:
X=pow(c1,a,p)
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder, (quotient, remainder) = remainder, divmod(lastremainder, remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder, lastx * (-1 if aa < 0 else 1), lasty * (-1 if bb < 0 else 1)
def modinv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError
    return x % m
Xinv=modinv(X,p)
texto_decifrado=(Xinv*c2)%p
print("Texto Descifrado =",texto_decifrado)

Texto Descifrado = 72797665
